In [87]:
import pandas as pd
import numpy as np 
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [88]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

In [89]:
df = pd.read_csv("df_dummies_f.csv")
#deleting col de sexo pois so temos feminino aqui 
df = df.drop('sexo', axis=1)
df.head()

,faixa_remuneracao_media_sm,Produção de lavouras temporárias,Horticultura e floricultura,Produção de lavouras permanentes,Produção de sementes e mudas certificadas,Pecuária,Atividades de apoio à agricultura e à pecuária,Caça e serviços relacionados,Produção florestal - florestas plantadas,Produção florestal - florestas nativas,Atividades de apoio à produção florestal,Pesca,Aqüicultura,Extração de carvão mineral,Extração de petróleo e gás natural,Extração de minério de ferro,Extração de minerais metálicos não-ferrosos,"Extração de pedra, areia e argila",Extração de outros minerais não-metálicos,Atividades de apoio à extração de petróleo e gás natural,"Atividades de apoio à extração de minerais, exceto petróleo e gás natural",Abate e fabricação de produtos de carne,Preservação do pescado e fabricação de produtos do pescado,"Fabricação de conservas de frutas, legumes e outros vegetais",Fabricação de óleos e gorduras vegetais e animais,Laticínios,"Moagem, fabricação de produtos amiláceos e de alimentos para animais",Fabricação e refino de açúcar,Torrefação e moagem de café,Fabricação de outros produtos alimentícios,Fabricação de bebidas alcoólicas,Fabricação de bebidas não-alcoólicas,Processamento industrial do fumo,Fabricação de produtos do fumo,Preparação e fiação de fibras têxteis,"Tecelagem, exceto malha",Fabricação de tecidos de malha,"Acabamentos em fios, tecidos e artefatos têxteis","Fabricação de artefatos têxteis, exceto vestuário",Confecção de artigos do vestuário e acessórios,Fabricação de artigos de malharia e tricotagem,Curtimento e outras preparações de couro,Fabricação de artigos para viagem e de artefatos diversos de couro,Fabricação de calçados,"Fabricação de partes para calçados, de qualquer material",Desdobramento de madeira,"Fabricação de produtos de madeira, cortiça e material trançado, exceto móveis",Fabricação de celulose e outras pastas para a fabricação de papel,"Fabricação de papel, cartolina e papel-cartão","Fabricação de embalagens de papel, cartolina, papel-cartão e papelão ondulado","Fabricação de produtos diversos de papel, cartolina, papel-cartão e papelão ondulado",Atividade de impressão,Serviços de pré-impressão e acabamentos gráficos,Reprodução de materiais gravados em qualquer suporte,Coquerias,Fabricação de produtos derivados do petróleo,Fabricação de biocombustíveis,Fabricação de produtos químicos inorgânicos,Fabricação de produtos químicos orgânicos,Fabricação de resinas e elastômeros,Fabricação de fibras artificiais e sintéticas,Fabricação de defensivos agrícolas e desinfestantes domissanitários,"Fabricação de sabões, detergentes, produtos de limpeza, cosméticos, produtos de perfumaria e de higiene pessoal","Fabricação de tintas, vernizes, esmaltes, lacas e produtos afins",Fabricação de produtos e preparados químicos diversos,Fabricação de produtos farmoquímicos,Fabricação de produtos farmacêuticos,Fabricação de produtos de borracha,Fabricação de produtos de material plástico,Fabricação de vidro e de produtos do vidro,Fabricação de cimento,"Fabricação de artefatos de concreto, cimento, fibrocimento, gesso e materiais semelhantes",Fabricação de produtos cerâmicos,Aparelhamento de pedras e fabricação de outros produtos de minerais não-metálicos,Produção de ferro-gusa e de ferroligas,Siderurgia,"Produção de tubos de aço, exceto tubos sem costura",Metalurgia dos metais não-ferrosos,Fundição,Fabricação de estruturas metálicas e obras de caldeiraria pesada,"Fabricação de tanques, reservatórios metálicos e caldeiras","Forjaria, estamparia, metalurgia do pó e serviços de tratamento de metais","Fabricação de artigos de cutelaria, de serralheria e ferramentas","Fabricação de equipamento bélico pesado, armas de fogo e munições",Fabricação de produtos de metal não especificados anteriormente,Fabricação de componentes eletrônicos,Fabricação de equipamentos de informática e periféricos,Fabricação de equipamentos de comunicação,"Fabricação de aparelhos de recepção, reprodução, gravação e amplificação de áu

## Correlacao de Pearson


In [90]:
df["faixa_remuneracao_media_sm"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [91]:
df_ultimas_faixas.shape

(108, 378)

In [92]:
df_primeiras_faixas=df[df['faixa_remuneracao_media_sm'].isin([0,1,2,3])] 
df_ultimas_faixas= df[df['faixa_remuneracao_media_sm'].isin([ 8, 9,10,11])] 
#df_ultima=df[df['faixa_remuneracao_media_sm'].isin([ 10,11])] 
dataframes = {
    "df_primeiras_faixas": df_primeiras_faixas,
    "df_ultimas_faixas": df_ultimas_faixas,

    
}

In [93]:

for nome, dataframe in dataframes.items(): 
    # Calculando a correlação de Pearson de todas as features com a variável alvo
    correlation_matrix = dataframe.corr()
    #print(dataframe.head())

    # Extrair as correlações da variável alvo e descartar a própria variável alvo da série
    target_correlation = correlation_matrix['faixa_remuneracao_media_sm'].drop('faixa_remuneracao_media_sm')

    # Ordenar as correlações em ordem decrescente de valor absoluto para identificar as mais significativas
    top_features_correlation = target_correlation.abs().sort_values(ascending=False)
    print(f"para o {nome} as 30 variaveis mais relevantes de acordo com a correlação de Pearson sao: ")
    # Mostrando as top 10 features mais correlacionadas com a variável alvo
    print(top_features_correlation.head(10))


para o df_primeiras_faixas as 30 variaveis mais relevantes de acordo com a correlação de Pearson sao: 
Atividades de vigilância, segurança privada e transporte de valores           0.634086
Comércio varejista de combustíveis para veículos automotores                  0.569853
Atividades de rádio                                                           0.539947
Atividades de televisão                                                       0.539692
Atividades de contabilidade, consultoria e auditoria contábil e tributária    0.526105
Instituição Financeira                                                        0.511251
Comércio varejista de material de construção                                  0.506201
Construção de edifícios                                                       0.492875
Comércio de veículos automotores                                              0.490042
65 OU MAIS                                                                    0.483369
Name: faixa_remuneracao_med

In [94]:
##DEPOIS COMPARAR COM O RESULTADO MASCULINO PARA AMBAS AS FAIXAS 

In [95]:
pd.set_option('display.max_rows', None)  # Para mostrar todas as linhas se necessário
pd.set_option('display.max_columns', None)  # Para mostrar todas as colunas
pd.set_option('display.max_colwidth', None)  # Para mostrar todo o conteúdo de cada célula sem truncar


In [96]:
for nome, dataframe in dataframes.items(): 
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']

    chi2_features = SelectKBest(chi2, k=10)
    X_kbest_features = chi2_features.fit_transform(X, y)

    # Obtendo os scores para as features selecionadas novamente
    chi2_scores = chi2_features.scores_

    # Criando um dataframe para visualizar os scores de cada feature
    chi2_scores_df = pd.DataFrame(chi2_scores, index=X.columns, columns=["Chi2 Score"]).sort_values(by="Chi2 Score", ascending=False)
    print(f"para o {nome} as 10 variaveis mais relevantes de acordo com o Chi2 Score sao: ")
    print(chi2_scores_df.head(10))
    


para o df_primeiras_faixas as 10 variaveis mais relevantes de acordo com o Chi2 Score sao: 
                                                                              Chi2 Score
Hotéis e similares                                                          14436.217083
Atividades de serviços de complementação diagnóstica e terapêutica          12400.439637
Comércio varejista de material de construção                                12278.772689
Comércio de peças e acessórios para veículos automotores                    11900.767119
Outras atividades de serviços pessoais                                      10424.347292
CLT U/ PF Ind                                                               10411.918857
Comércio varejista de artigos culturais, recreativos e esportivos           10307.908298
Comércio de veículos automotores                                            10119.816408
Atividades de contabilidade, consultoria e auditoria contábil e tributária   9761.574854
Instituição Financ

## RFE

In [97]:

for nome, dataframe in dataframes.items(): 
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']
    model = LinearRegression()
    rfe = RFE(estimator=model, n_features_to_select=30)
    rfe.fit(X, y)
    selected_features_rfe = X.columns[rfe.support_]
    print(f"para o {nome} as 10 variaveis mais relevantes de acordo com o rfe sao: ")
    print(selected_features_rfe)

para o df_primeiras_faixas as 10 variaveis mais relevantes de acordo com o rfe sao: 
Index(['Atividades de apoio à agricultura e à pecuária',
       'Fabricação de conservas de frutas, legumes e outros vegetais',
       'Fabricação de óleos e gorduras vegetais e animais',
       'Preparação e fiação de fibras têxteis', 'Tecelagem, exceto malha',
       'Curtimento e outras preparações de couro', 'Fabricação de cimento',
       'Fabricação de produtos cerâmicos',
       'Aparelhamento de pedras e fabricação de outros produtos de minerais não-metálicos',
       'Siderurgia',
       'Fabricação de equipamentos e aparelhos elétricos não especificados anteriormente',
       'Recondicionamento e recuperação de motores para veículos automotores',
       'Fabricação de móveis',
       'Fabricação de artigos de joalheria, bijuteria e semelhantes',
       'Construção de rodovias, ferrovias, obras urbanas e obras de arte especiais',
       'Comércio atacadista de matérias-primas agrícolas e anima

KeyboardInterrupt: 

## lasso 

In [ ]:

from sklearn.linear_model import LassoCV
for nome, dataframe in dataframes.items():
    X = dataframe.drop('faixa_remuneracao_media_sm', axis=1)
    y = dataframe['faixa_remuneracao_media_sm']
    lasso = LassoCV(cv=5).fit(X, y)
    important_features_lasso = np.where(lasso.coef_ != 0)[0]
    print(f"Para o {nome}, as variáveis mais relevantes de acordo com o Lasso são:")
    
    # Iterando sobre os índices das características importantes e imprimindo cada nome de variável em uma nova linha
    for index in important_features_lasso:
        print(X.columns[index])

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.287e-02, tolerance: 1.075e-02
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.295e-02, tolerance: 1.075e-02
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.409e-02, toleranc

Para o df_primeiras_faixas, as variáveis mais relevantes de acordo com o Lasso são:
Construção de edifícios
Comércio de veículos automotores
Comércio de peças e acessórios para veículos automotores
Comércio varejista de produtos alimentícios, bebidas e fumo
Comércio varejista de combustíveis para veículos automotores
Comércio varejista de material de construção
Comércio varejista de equipamentos de informática e comunicação
Comércio varejista de produtos farmacêuticos, perfumaria e cosméticos e artigos médicos, ópticos e ortopédicos
Comércio varejista de produtos novos não especificados anteriormente e de produtos usados
Hotéis e similares
Planos de saúde
Atividades de contabilidade, consultoria e auditoria contábil e tributária
Atividades de vigilância, segurança privada e transporte de valores
Atividades de teleatendimento
Serviços coletivos prestados pela administração pública
Educação superior
Atividades de serviços de complementação diagnóstica e terapêutica
Atividades de profissi

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.738e-02, tolerance: 1.075e-02
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.586e-02, tolerance: 1.075e-02
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.696e-02, toleranc

Para o df_ultimas_faixas, as variáveis mais relevantes de acordo com o Lasso são:
Fabricação de produtos farmacêuticos
Geração, transmissão e distribuição de energia elétrica
Intermediação monetária - depósitos à vista
Administração do estado e da política econômica e social
Serviços coletivos prestados pela administração pública
Seguridade social obrigatória
Ensino médio
Educação superior
Educação profissional de nível técnico e tecnológico
Outras atividades de ensino
Atividades de atendimento hospitalar
Atividades de apoio à gestão de saúde
Extrativa Mineral
Alimentos e Bebidas
Comércio Varejista
Comércio Atacadista
Instituição Financeira
Adm Técnica Profissional
Transporte e Comunicações
Aloj Comunic
Ensino
Administração Pública
CLT U/ PJ Ind
Estatutário
Estatutário RGPS
Estatutário não Efetivo
Médio Completo
Superior Incompleto
Superior Completo
Mestrado
Doutorado
18 A 24
25 A 29
30 A 39
40 A 49
65 OU MAIS


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.539e-03, tolerance: 1.074e-03
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.339e-03, tolerance: 1.074e-03
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.721e-03, toleranc

Para o df_ultima, as variáveis mais relevantes de acordo com o Lasso são:
Ensino


/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.176e-02, tolerance: 1.100e-03
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.721e-02, tolerance: 1.100e-03
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.593e-02, toleranc

Analise final :